In [1]:
from nsetools import Nse
import yfinance as yf
import pandas as pd
import openpyxl
import talib as ta
nse = Nse()

In [2]:
# Get all stock codes

all_stock_codes = nse.get_stock_codes()
keys = list(all_stock_codes.keys())[1:-1]
keys = [sub + '.NS' for sub in keys]

In [3]:
# List of strategy

# 1. Check 44ma rising(1d)
# 2. 44MA(1d,1w)
# 3. 44MA+200MA(1d,1w)
# 4. BollingerBand(1d,1w)

In [4]:
def get_data(tickers, period, interval):
    
    data = yf.download(tickers=tickers,period=period,interval=interval,progress=False)
    
    
    # Enable this if running for previous day before EOD
    prev_day = False
    if prev_day == True:
        data.drop(index=data.index[-1], axis=0, inplace=True)
        
    
    data['44MA'] = ta.SMA(data['Close'],44)
    data['200MA'] = ta.SMA(data['Close'],200)
    data['50MA'] = ta.SMA(data['Close'],50)
    data['up_band'], data['mid_band'], data['low_band'] = ta.BBANDS(data['Close'], timeperiod =20)
    data = data.drop(['mid_band'],axis=1)
    

    
    return data

In [5]:
# Check if 44MA is rising for last n instances in dataframe: data

def check_44ma_rising(data,n):
    
    flag_44ma_rising = True
    for x in range(n):
        x +=1
        try:
            flag_44ma_rising = ((data['44MA'].iloc[-1*x] > data['44MA'].iloc[(-1*x)-1]) and flag_44ma_rising)
        except:
            flag_44ma_rising = False

    return flag_44ma_rising

In [6]:
def check_44ma_str(data):
    
    # Check 44MA increasing for 3 points
    flag_1 = check_44ma_rising(data,3)
    
    # Check for green candle
    if flag_1 == True:
        flag_2 = data['Close'].iloc[-1] > data['Open'].iloc[-1]
    else:
        flag_2 = False
    
    # Check for low to be close to bb_low_band
    if (flag_1 and flag_2) == True:
        flag_3 = (abs(data['Low'].iloc[-1] - data['44MA'].iloc[-1])/(data['44MA'].iloc[-1])) < 0.02
    else:
        flag_3 = False
    
    
    flag_44ma_str = ((flag_1 and flag_2) and flag_3)
    
    return flag_44ma_str

In [7]:
# n is max number of candles in W pattern

def check_bb_low_str(data,n):
    
    flag_1 = flag_2 = flag_3 = flag_4 = flag_4_1 = flag_5 = flag_6_1 = flag_6_2 = flag_6 = flag_bb_low_str = False # Initialize all the flags to false
    add = mean = low = min_low = 0
    
    # Check if the last candle is green (bullish)
    flag_1 = (data['Close'].iloc[-1] > data['Open'].iloc[-1])
    
    # Check if the last candle is red hammer
    delta = (data['High'].iloc[-1] - data['Low'].iloc[-1])
    flag_2 = (data['Close'].iloc[-1] > (data['Low'].iloc[-1] + 0.5*delta)) and (data['Close'].iloc[-1] < data['Open'].iloc[-1])
    
    # Check if the low of last candle is close to bb_low_band
    if (flag_1 or flag_2) == True:
        flag_3 = ((abs(data['Low'].iloc[-1] - data['low_band'].iloc[-1])/(data['low_band'].iloc[-1])) < 0.10)
    
    if flag_3 == True:
        
        # Calculate support band
        z = 0.05 # x+-z will be used to calculate support band
        support_low = (1-z)*data['Low'].iloc[-1]
        support_high = (1+z)*data['Low'].iloc[-1]
    
        # Find a candle in last n instances which has closed below bb_low_band
        for x in range(n):
            instance = (-1*x)-2
            flag_4 = (data['Close'].iloc[instance] < data['low_band'].iloc[instance]) # Set flag to true if a valid candle is found
            low = data['Low'].iloc[instance]
            
            if min_low == 0:
                min_low = low
            elif (low < min_low) and (flag_4 == False):
                min_low = low
            
            if flag_4 == True:
                break
            else:
                flag_4 = False # Set flag to false if a valid candle is not found
                
        flag_4_1 = (min_low > (0.95* min(data['Low'].iloc[-1],data['Low'].iloc[instance])) )
        #print(min_low,data['Low'].iloc[-1])
                
    if (flag_4 == True) and (flag_4_1 == True) and (x > 1):
        for y in range(x):
            instance_2 = (-1*x)-2
            add = add + data['High'].iloc[instance_2]
        mean = add / (x-1)
        flag_5 = True
    
    
    # Check if the x instance has close or low value withing support band as per last candle
    if flag_5 == True:
        flag_6_1 = (data['Close'].iloc[instance] < support_high) and (data['Close'].iloc[instance] > support_low)
        flag_6_2 = (data['Low'].iloc[instance] < support_high) and (data['Low'].iloc[instance] > support_low)
        flag_6 = flag_6_1 or flag_6_2
        
    # Check if the average of in-between candles is greater than support
    if flag_6 == True:
        flag_bb_low_str = (mean > support_low)
        
#     print('flag_1:',flag_1,' flag_2:',flag_2,' flag_3:',flag_3,' flag_4:',flag_4,' flag_4_1:',flag_4_1,' flag_5:',flag_5,
#           ' flag_6_1:',flag_6_1,' flag_6_2:',flag_6_2,' flag_6:',flag_6,' flag_bb_low_str:',flag_bb_low_str)
    
    return(flag_bb_low_str)

In [ ]:
len(keys)

In [8]:
# Run this for 44MA Strategy

results_44ma = pd.DataFrame(columns=['Stock','44ma_str'])
m = 0
i=0
for stock in keys:
    
    tickers = stock
    period = "3mo"
    interval = "1d"
    i+=1
    #print("{0} {1}".format(i,stock))
    try:
        data = get_data(tickers,period,interval)
    except:
        continue
    
    # 44MA rising check    
#     flag_44ma_rising = check_44ma_rising(data,5)
#     if flag_44ma_rising==True:
#         results.loc[m]=[tickers,flag_44ma_rising]
#         m +=1
        
    # 44MA strategy check
    flag_44ma_str = check_44ma_str(data)
    if flag_44ma_str==True:
        results_44ma.loc[m]=[tickers,flag_44ma_str]
        m +=1
        
print(results_44ma)


1 Failed download:
- DIGJAMLMTD.NS: No data found for this date range, symbol may be delisted

1 Failed download:
- EMAMIPAP.NS: No data found for this date range, symbol may be delisted

1 Failed download:
- NIRAJISPAT.NS: No data found for this date range, symbol may be delisted

1 Failed download:
- SUBCAPCITY.NS: No data found for this date range, symbol may be delisted

1 Failed download:
- ZODIAC.NS: No data found for this date range, symbol may be delisted
            Stock  44ma_str
0       AIAENG.NS      True
1    ANDHRAPAP.NS      True
2   APOLLOTYRE.NS      True
3    ARCHIDPLY.NS      True
4     ASHOKLEY.NS      True
5       AUBANK.NS      True
6   BAJAJ-AUTO.NS      True
7   BANDHANBNK.NS      True
8          BDL.NS      True
9     CINELINE.NS      True
10      COLPAL.NS      True
11   CREDITACC.NS      True
12      CRISIL.NS      True
13    EIDPARRY.NS      True
14    FMGOETZE.NS      True
15        GAEL.NS      True
16  GENUSPOWER.NS      True
17      GESHIP.NS      True

In [9]:
# Run this for BB Strategy

results_bb = pd.DataFrame(columns=['Stock','bb_low_str'])
m = 0

for stock in keys:
    
    tickers = stock
    period = "3mo"
    interval = "1d"
    n = 20
    
    try:
        data = get_data(tickers,period,interval)
    except:
        continue
        
    if len(data) <= n:
        continue
        
    # BB low strategy check
    flag_bb_low_str = check_bb_low_str(data,n)
    if flag_bb_low_str==True:
        results_bb.loc[m]=[tickers,flag_bb_low_str]
        
        print(results_bb['Stock'].loc[m])
        m +=1
        
print(results_bb)

20MICRONS.NS
ACCURACY.NS
AJANTPHARM.NS
ALLCARGO.NS
ANUP.NS
APLLTD.NS
ARSSINFRA.NS

1 Failed download:
- ARTEMISMED.NS: No data found for this date range, symbol may be delisted
ARVSMART.NS
ASALCBR.NS
BAJAJ-AUTO.NS
BALKRISHNA.NS
BALRAMCHIN.NS
BDL.NS
BIRLACORPN.NS
BOROLTD.NS
CGPOWER.NS
CHALET.NS
CIGNITITEC.NS
CIPLA.NS
COUNCODOS.NS
CREATIVEYE.NS
CUBEXTUB.NS
CUMMINSIND.NS
DATAPATTNS.NS
DBSTOCKBRO.NS
DHANUKA.NS

1 Failed download:
- DIGJAMLMTD.NS: No data found for this date range, symbol may be delisted
DODLA.NS
DRREDDY.NS
EICHERMOT.NS

1 Failed download:
- EMAMIPAP.NS: No data found for this date range, symbol may be delisted
ESCORTS.NS
GRPLTD.NS
GUFICBIO.NS

1 Failed download:
- GUJRAFFIA.NS: No data found for this date range, symbol may be delisted
GULPOLY.NS
HAVELLS.NS
HCC.NS
HFCL.NS
HIKAL.NS
ICICIGI.NS
IGL.NS
IGPL.NS
IMAGICAA.NS
INDNIPPON.NS
INDOBORAX.NS
INDTERRAIN.NS
INTLCONV.NS
IONEXCHANG.NS
ISFT.NS
JAYAGROGN.NS
JAYNECOIND.NS
JMA.NS
JYOTHYLAB.NS
KALPATPOWR.NS
KDDL.NS
KIMS.NS
KIRLOSE

In [10]:
# # Back testing BB startegy

# data = yf.download(tickers='HINDALCO.NS',period='1y',interval='1d',progress=False)
# data['up_band'], data['mid_band'], data['low_band'] = ta.BBANDS(data['Close'], timeperiod =20)

# for x in range(100):
    
#     flag_bb_low_str = check_bb_low_str(data,18)
    
#     if flag_bb_low_str == True:
#         print(data.index[-1])
        
#     #print(data.index[-1],flag_bb_low_str)
#     data.drop(index=data.index[-1], axis=0, inplace=True)
    


In [24]:
results_44ma = results_44ma.drop(['44ma_str'],axis=1)
results_bb = results_bb.drop(['bb_low_str'],axis=1)

with pd.ExcelWriter('output.xlsx') as writer:
        results_44ma.to_excel(writer, sheet_name='44ma')
        results_bb.to_excel(writer, sheet_name='bb')

In [ ]:
results_44ma